# RNN

In [ ]:
keras.layers.RNN(cell, 
                 return_sequences=False, 
                 return_state=False, 
                 go_backwards=False, 
                 stateful=False, 
                 unroll=False)

循环神经网络层基类。

### 参数

cell: 一个 RNN 单元实例。RNN 单元是一个具有以下项目的类：

一个 call(input_at_t, states_at_t) 方法， 它返回  (output_at_t, states_at_t_plus_1)。 单元的调用方法也可以采用可选参数 constants， 详见下面的小节 "关于传递外部常量的注意事项"。
一个 state_size 属性。这可以是单个整数（单个状态）， 在这种情况下，它是循环层状态的大小（应该与单元输出的大小相同）。 这也可以是整数的列表/元组（每个状态一个大小）。 在这种情况下，第一项（state_size [0]）应该与单元输出的大小相同。 cell 也可能是 RNN 单元实例的列表，在这种情况下，RNN 的单元将堆叠在另一个单元上，实现高效的堆叠 RNN。

return_sequences: 布尔值。是返回输出序列中的最后一个输出，还是全部序列。

return_state: 布尔值。除了输出之外是否返回最后一个状态。

go_backwards: 布尔值 (默认 False)。 如果为 True，则向后处理输入序列并返回相反的序列。

stateful: 布尔值 (默认 False)。 如果为 True，则批次中索引 i 处的每个样品的最后状态将用作下一批次中索引 i 样品的初始状态。

unroll: 布尔值 (默认 False)。 如果为 True，则网络将展开，否则将使用符号循环。 展开可以加速 RNN，但它往往会占用更多的内存。 展开只适用于短序列。

input_dim: 输入的维度（整数）。 将此层用作模型中的第一层时，此参数（或者，关键字参数 input_shape）是必需的。

input_length: 输入序列的长度，在恒定时指定。 如果你要在上游连接 Flatten 和 Dense 层， 则需要此参数（如果没有它，无法计算全连接输出的尺寸）。 请注意，如果循环神经网络层不是模型中的第一层， 则需要在第一层的层级指定输入长度（例如，通过 input_shape 参数）。
### 输入尺寸

3D 张量，尺寸为 (batch_size, timesteps, input_dim)。

### 输出尺寸

如果 return_state 为 True，则返回张量列表。 第一个张量为输出。剩余的张量为最后的状态， 每个张量的尺寸为 (batch_size, units)。

否则，返回尺寸为 (batch_size, units) 的 2D 张量。
### 屏蔽覆盖

该层支持以可变数量的时间步长对输入数据进行屏蔽覆盖。 要将屏蔽引入数据，请使用 Embedding 层， 并将mask_zero 参数设置为 True。

关于在 RNN 中使用状态的注意事项

你可以将 RNN 层设置为 stateful（有状态的）， 这意味着针对一批中的样本计算的状态将被重新用作下一批样品的初始状态。 这假定在不同连续批次的样品之间有一对一的映射。

为了使状态有效： - 在层构造器中指定 stateful=True。 - 为你的模型指定一个固定的批次大小， 如果是顺序模型，为你的模型的第一层传递一个  batch_input_shape=(...) 参数。 - 为你的模型指定一个固定的批次大小， 如果是顺序模型，为你的模型的第一层传递一个 batch_input_shape=(...)。 如果是带有 1 个或多个 Input 层的函数式模型，为你的模型的所有第一层传递一个 batch_shape=(...)。 这是你的输入的预期尺寸，包括批量维度。 它应该是整数的元组，例如  (32, 10, 100)。 - 在调用 fit() 是指定 shuffle=False。

要重置模型的状态，请在特定图层或整个模型上调用 .reset_states()。

### 关于指定 RNN 初始状态的注意事项

您可以通过使用关键字参数 initial_state 调用它们来符号化地指定 RNN 层的初始状态。 initial_state 的值应该是表示 RNN 层初始状态的张量或张量列表。

您可以通过调用带有关键字参数 states 的 reset_states 方法来数字化地指定 RNN 层的初始状态。 states 的值应该是一个代表 RNN 层初始状态的 Numpy 数组或者 Numpy 数组列表。

### 关于给 RNN 传递外部常量的注意事项

你可以使用 RNN.__call__ （以及 RNN.call）的 constants 关键字参数将「外部」常量传递给单元。 这要求 cell.call 方法接受相同的关键字参数 constants。 这些常数可用于调节附加静态输入（不随时间变化）上的单元转换，也可用于注意力机制。

In [ ]:
#首先，定义一个RNN单元，作为网络层子类

class MimimalRNNCell(keras.layers.Layer):
    def __init__(self,units,**kwargs):
        self.units=units
        self.state_size=units
        super(MinimalRNNCell,self).__init__(**kwargs)
        
    def build(self,input_shape):
        self.kernel=self.add_weight(shape=(input_shape[-1],self.units),
                                    initializer='uniform',
                                    name='kernel')
        self.recurrent_kernel=self.add_weight(
            shape=(self.units,self.units),
            initializer='uniform',
            name='recurrent_kernel')
        self.built=True
    
    def call(self,inputs,states):
        prev_output=atates[0]
        h=K.dot(inputs,self.kernel)
        output=h+K.dot(prev_output,self.recurrent_kernel)
        return output,[output]

#在RNN层使用这个单元

cell=MinimalRNNCell(32)
x=keras.Input((None,5))
layer=RNN(cell)
y=layer(x)

#使用单元格构建堆叠的RNN的方法：

cells=[MinimalRNNCell(32),minimalRNNCell(64)]
x=keras.Input((None,5))
layer=RNN(cells)
y=layer(x)

# SimpleRNN

In [ ]:
keras.layers.SimpleRNN(units, 
                       activation='tanh', 
                       use_bias=True, 
                       kernel_initializer='glorot_uniform', 
                       recurrent_initializer='orthogonal', 
                       bias_initializer='zeros', 
                       kernel_regularizer=None, 
                       recurrent_regularizer=None, 
                       bias_regularizer=None, 
                       activity_regularizer=None, 
                       kernel_constraint=None, 
                       recurrent_constraint=None, 
                       bias_constraint=None, 
                       dropout=0.0, 
                       recurrent_dropout=0.0, 
                       return_sequences=False, 
                       return_state=False, 
                       go_backwards=False, 
                       stateful=False, unroll=False)

完全连接的 RNN，其输出将被反馈到输入。

### 参数

units: 正整数，输出空间的维度。

activation: 要使用的激活函数 (详见 activations)。 如果传入 None，则不使用激活函数 (即 线性激活：a(x) = x)。

use_bias: 布尔值，该层是否使用偏置向量。

kernel_initializer: kernel 权值矩阵的初始化器， 用于输入的线性转换 (详见 initializers)。

recurrent_initializer: recurrent_kernel 权值矩阵 的初始化器，用于循环层状态的线性转换 (详见 initializers)。

bias_initializer:偏置向量的初始化器 (详见initializers).

kernel_regularizer: 运用到 kernel 权值矩阵的正则化函数 (详见 regularizer)。

recurrent_regularizer: 运用到 recurrent_kernel 权值矩阵的正则化函数 (详见 regularizer)。

bias_regularizer: 运用到偏置向量的正则化函数 (详见 regularizer)。

activity_regularizer: 运用到层输出（它的激活值）的正则化函数 (详见 regularizer)。

kernel_constraint: 运用到 kernel 权值矩阵的约束函数 (详见 constraints)。

recurrent_constraint: 运用到 recurrent_kernel 权值矩阵的约束函数 (详见 constraints)。

bias_constraint: 运用到偏置向量的约束函数 (详见 constraints)。

dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于输入的线性转换。

recurrent_dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于循环层状态的线性转换。

return_sequences: 布尔值。是返回输出序列中的最后一个输出，还是全部序列。

return_state: 布尔值。除了输出之外是否返回最后一个状态。

go_backwards: 布尔值 (默认 False)。 如果为 True，则向后处理输入序列并返回相反的序列。

stateful: 布尔值 (默认 False)。 如果为 True，则批次中索引 i 处的每个样品 的最后状态将用作下一批次中索引 i 样品的初始状态。

unroll: 布尔值 (默认 False)。 如果为 True，则网络将展开，否则将使用符号循环。 展开可以加速 RNN，但它往往会占用更多的内存。 展开只适用于短序列。

# GRU

In [ ]:
keras.layers.GRU(units, 
                 activation='tanh', 
                 recurrent_activation='hard_sigmoid', 
                 use_bias=True, 
                 kernel_initializer='glorot_uniform', 
                 recurrent_initializer='orthogonal', 
                 bias_initializer='zeros', 
                 kernel_regularizer=None, 
                 recurrent_regularizer=None, 
                 bias_regularizer=None, 
                 activity_regularizer=None, 
                 kernel_constraint=None, 
                 recurrent_constraint=None, 
                 bias_constraint=None, 
                 dropout=0.0, 
                 recurrent_dropout=0.0, 
                 implementation=1, 
                 return_sequences=False, 
                 return_state=False, 
                 go_backwards=False, 
                 stateful=False, 
                 unroll=False, 
                 reset_after=False)

门限循环单元网络 - Cho et al. 2014.

有两种变体。默认的基于 1406.1078v3 并且在矩阵乘法之前将复位门应用于隐藏状态。 另一种则是基于 1406.1078v1 并且顺序倒置。

第二种变体与 CuDNNGRU(GPU-only) 兼容并且允许在 CPU 上进行推理。 因此它对于  kernel 和 recurrent_kernel 有可分离偏置。 使用 'reset_after'=True 和  recurrent_activation='sigmoid' 。

### 参数

### 参数

units: 正整数，输出空间的维度。

activation: 要使用的激活函数 (详见 activations)。 默认：双曲正切 (tanh)。 如果传入 None，则不使用激活函数 (即 线性激活：a(x) = x)。

recurrent_activation: 用于循环时间步的激活函数 (详见 activations)。 默认：分段线性近似 sigmoid (hard_sigmoid)。 如果传入 None，则不使用激活函数 (即 线性激活：a(x) = x)。

use_bias: 布尔值，该层是否使用偏置向量。

kernel_initializer: kernel 权值矩阵的初始化器， 用于输入的线性转换 (详见 initializers)。

recurrent_initializer: recurrent_kernel 权值矩阵 的初始化器，用于循环层状态的线性转换 (详见 initializers)。

bias_initializer:偏置向量的初始化器 (详见initializers).

kernel_regularizer: 运用到 kernel 权值矩阵的正则化函数 (详见 regularizer)。

recurrent_regularizer: 运用到 recurrent_kernel 权值矩阵的正则化函数 (详见 regularizer)。

bias_regularizer: 运用到偏置向量的正则化函数 (详见 regularizer)。

activity_regularizer: 运用到层输出（它的激活值）的正则化函数 (详见 regularizer)。

kernel_constraint: 运用到 kernel 权值矩阵的约束函数 (详见 constraints)。

recurrent_constraint: 运用到 recurrent_kernel 权值矩阵的约束函数 (详见 constraints)。

bias_constraint: 运用到偏置向量的约束函数 (详见 constraints)。

dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于输入的线性转换。

recurrent_dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于循环层状态的线性转换。

implementation: 实现模式，1 或 2。 模式 1 将把它的操作结构化为更多的小的点积和加法操作， 而模式 2 将把它们分批到更少，更大的操作中。 这些模式在不同的硬件和不同的应用中具有不同的性能配置文件。

return_sequences: 布尔值。是返回输出序列中的最后一个输出，还是全部序列。

return_state: 布尔值。除了输出之外是否返回最后一个状态。

go_backwards: 布尔值 (默认 False)。 如果为 True，则向后处理输入序列并返回相反的序列。

stateful: 布尔值 (默认 False)。 如果为 True，则批次中索引 i 处的每个样品的最后状态 将用作下一批次中索引 i 样品的初始状态。

unroll: 布尔值 (默认 False)。 如果为 True，则网络将展开，否则将使用符号循环。 展开可以加速 RNN，但它往往会占用更多的内存。 展开只适用于短序列。

reset_after: GRU 公约 (是否在矩阵乘法之前或者之后使用重置门)。 False =「之前」(默认)，Ture =「之后」( CuDNN 兼容)。

# LSTM

In [ ]:
keras.layers.LSTM(units, 
                  activation='tanh', 
                  recurrent_activation='hard_sigmoid', 
                  use_bias=True, 
                  kernel_initializer='glorot_uniform', 
                  recurrent_initializer='orthogonal', 
                  bias_initializer='zeros', 
                  unit_forget_bias=True, 
                  kernel_regularizer=None, 
                  recurrent_regularizer=None, 
                  bias_regularizer=None, 
                  activity_regularizer=None, 
                  kernel_constraint=None, 
                  recurrent_constraint=None, 
                  bias_constraint=None, 
                  dropout=0.0, 
                  recurrent_dropout=0.0, 
                  implementation=1, 
                  return_sequences=False, 
                  return_state=False, 
                  go_backwards=False, 
                  stateful=False, 
                  unroll=False)

长短期记忆网络层 - Hochreiter 1997.

### 参数

units: 正整数，输出空间的维度。

activation: 要使用的激活函数 (详见 activations)。 如果传入 None，则不使用激活函数 (即 线性激活：a(x) = x)。

recurrent_activation: 用于循环时间步的激活函数 (详见 activations)。

use_bias: 布尔值，该层是否使用偏置向量。

kernel_initializer: kernel 权值矩阵的初始化器， 用于输入的线性转换 (详见 initializers)。

recurrent_initializer: recurrent_kernel 权值矩阵 的初始化器，用于循环层状态的线性转换 (详见 initializers)。

bias_initializer:偏置向量的初始化器 (详见initializers).

unit_forget_bias: 布尔值。 如果为 True，初始化时，将忘记门的偏置加 1。 将其设置为 True 同时还会强制 bias_initializer="zeros"。 这个建议来自 Jozefowicz et al.。

kernel_regularizer: 运用到 kernel 权值矩阵的正则化函数 (详见 regularizer)。

recurrent_regularizer: 运用到 recurrent_kernel 权值矩阵的正则化函数 (详见 regularizer)。

bias_regularizer: 运用到偏置向量的正则化函数 (详见 regularizer)。

activity_regularizer: 运用到层输出（它的激活值）的正则化函数 (详见 regularizer)。

kernel_constraint: 运用到 kernel 权值矩阵的约束函数 (详见 constraints)。

recurrent_constraint: 运用到 recurrent_kernel 权值矩阵的约束函数 (详见 constraints)。

bias_constraint: 运用到偏置向量的约束函数 (详见 constraints)。

dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于输入的线性转换。

recurrent_dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于循环层状态的线性转换。

implementation: 实现模式，1 或 2。 模式 1 将把它的操作结构化为更多的小的点积和加法操作， 而模式 2 将把它们分批到更少，更大的操作中。 这些模式在不同的硬件和不同的应用中具有不同的性能配置文件。

return_sequences: 布尔值。是返回输出序列中的最后一个输出，还是全部序列。

return_state: 布尔值。除了输出之外是否返回最后一个状态。

go_backwards: 布尔值 (默认 False)。 如果为 True，则向后处理输入序列并返回相反的序列。

stateful: 布尔值 (默认 False)。 如果为 True，则批次中索引 i 处的每个样品的最后状态 将用作下一批次中索引 i 样品的初始状态。

unroll: 布尔值 (默认 False)。 如果为 True，则网络将展开，否则将使用符号循环。 展开可以加速 RNN，但它往往会占用更多的内存。 展开只适用于短序列。

# ConvLSTM2D

In [ ]:
keras.layers.ConvLSTM2D(filters, 
                        kernel_size, 
                        strides=(1, 1), 
                        padding='valid', 
                        data_format=None, 
                        dilation_rate=(1, 1), 
                        activation='tanh', 
                        recurrent_activation='hard_sigmoid', 
                        use_bias=True, 
                        kernel_initializer='glorot_uniform', 
                        recurrent_initializer='orthogonal', 
                        bias_initializer='zeros', 
                        unit_forget_bias=True, 
                        kernel_regularizer=None, 
                        recurrent_regularizer=None, 
                        bias_regularizer=None, 
                        activity_regularizer=None, 
                        kernel_constraint=None, 
                        recurrent_constraint=None, 
                        bias_constraint=None, 
                        return_sequences=False, 
                        go_backwards=False, 
                        stateful=False, 
                        dropout=0.0, 
                        recurrent_dropout=0.0)

卷积 LSTM.

它类似于 LSTM 层，但输入变换和循环变换都是卷积的。

### 参数

filters: 整数，输出空间的维度 （即卷积中滤波器的输出数量）。

kernel_size: 一个整数，或者单个整数表示的元组或列表， 指明 1D 卷积窗口的长度。

strides: 一个整数，或者单个整数表示的元组或列表， 指明卷积的步长。 指定任何 stride 值 != 1 与指定 dilation_rate 值 != 1 两者不兼容。

padding: "valid" 或 "same" 之一 (大小写敏感)。

data_format: 字符串， channels_last (默认) 或 channels_first 之一。 输入中维度的顺序。 channels_last 对应输入尺寸为 (batch, height, width, channels)， channels_first 对应输入尺寸为 (batch, channels, height, width)。 它默认为从 Keras 配置文件 ~/.keras/keras.json 中 找到的 image_data_format 值。 如果你从未设置它，将使用 "channels_last"。

dilation_rate: 一个整数，或 n 个整数的元组/列表，指定用于膨胀卷积的膨胀率。 目前，指定任何 dilation_rate 值 != 1 与指定 stride 值 != 1 两者不兼容。

activation: 要使用的激活函数 (详见 activations)。 如果传入 None，则不使用激活函数 (即 线性激活：a(x) = x)。

recurrent_activation: 用于循环时间步的激活函数 (详见 activations)。

use_bias: 布尔值，该层是否使用偏置向量。

kernel_initializer: kernel 权值矩阵的初始化器， 用于输入的线性转换 (详见 initializers)。

recurrent_initializer: recurrent_kernel 权值矩阵 的初始化器，用于循环层状态的线性转换 (详见 initializers)。

bias_initializer:偏置向量的初始化器 (详见initializers).

unit_forget_bias: 布尔值。 如果为 True，初始化时，将忘记门的偏置加 1。 将其设置为 True 同时还会强制 bias_initializer="zeros"。 这个建议来自 Jozefowicz et al.。

kernel_regularizer: 运用到 kernel 权值矩阵的正则化函数 (详见 regularizer)。

recurrent_regularizer: 运用到 recurrent_kernel 权值矩阵的正则化函数 (详见 regularizer)。

bias_regularizer: 运用到偏置向量的正则化函数 (详见 regularizer)。

activity_regularizer: 运用到层输出（它的激活值）的正则化函数 (详见 regularizer)。

kernel_constraint: 运用到 kernel 权值矩阵的约束函数 (详见 constraints)。

recurrent_constraint: 运用到 recurrent_kernel 权值矩阵的约束函数 (详见 constraints)。

bias_constraint: 运用到偏置向量的约束函数 (详见 constraints)。

return_sequences: 布尔值。是返回输出序列中的最后一个输出，还是全部序列。

go_backwards: 布尔值 (默认 False)。 如果为 True，则向后处理输入序列并返回相反的序列。

stateful: 布尔值 (默认 False)。 如果为 True，则批次中索引 i 处的每个样品的最后状态 将用作下一批次中索引 i 样品的初始状态。

dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于输入的线性转换。

recurrent_dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于循环层状态的线性转换。
### 输入尺寸

如果 data_format='channels_first'， 返回 5D 张量，尺寸为： (samples,time, channels, rows, cols)。

如果 data_format='channels_last'， 返回 5D 张量，尺寸为： (samples,time, rows, cols, channels)。
### 输出尺寸

如果 return_sequences，
如果 data_format='channels_first'， 返回 5D 张量，尺寸为： (samples, time, filters, output_row, output_col)。


如果 data_format='channels_last'， 返回 5D 张量，尺寸为： (samples, time, output_row, output_col, filters)。

否则，
如果 data_format ='channels_first'， 返回 4D 张量，尺寸为： (samples, filters, output_row, output_col)。

如果 data_format='channels_last'， 返回 4D 张量，尺寸为： (samples, output_row, output_col, filters)。 o_row 和 o_col 依赖于过滤器的尺寸和填充。
### 异常

ValueError: 无效的构造参数。

# ConvLSTM2D

In [ ]:
keras.layers.ConvLSTM2D(filters, 
                        kernel_size, 
                        strides=(1, 1), 
                        padding='valid', 
                        data_format=None, 
                        dilation_rate=(1, 1), 
                        activation='tanh', 
                        recurrent_activation='hard_sigmoid', 
                        use_bias=True, 
                        kernel_initializer='glorot_uniform', 
                        recurrent_initializer='orthogonal', 
                        bias_initializer='zeros', 
                        unit_forget_bias=True, 
                        kernel_regularizer=None, 
                        recurrent_regularizer=None, 
                        bias_regularizer=None, 
                        activity_regularizer=None, 
                        kernel_constraint=None, 
                        recurrent_constraint=None, 
                        bias_constraint=None, 
                        return_sequences=False, 
                        go_backwards=False, 
                        stateful=False, 
                        dropout=0.0, 
                        recurrent_dropout=0.0)

卷积 LSTM.

它类似于 LSTM 层，但输入变换和循环变换都是卷积的。

### 参数

filters: 整数，输出空间的维度 （即卷积中滤波器的输出数量）。

kernel_size: 一个整数，或者单个整数表示的元组或列表， 指明 1D 卷积窗口的长度。

strides: 一个整数，或者单个整数表示的元组或列表， 指明卷积的步长。 指定任何 stride 值 != 1 与指定 dilation_rate 值 != 1 两者不兼容。

padding: "valid" 或 "same" 之一 (大小写敏感)。

data_format: 字符串， channels_last (默认) 或 channels_first 之一。 输入中维度的顺序。 channels_last 对应输入尺寸为 (batch, height, width, channels)， channels_first 对应输入尺寸为 (batch, channels, height, width)。 它默认为从 Keras 配置文件 ~/.keras/keras.json 中 找到的 image_data_format 值。 如果你从未设置它，将使用 "channels_last"。

dilation_rate: 一个整数，或 n 个整数的元组/列表，指定用于膨胀卷积的膨胀率。 目前，指定任何 dilation_rate 值 != 1 与指定 stride 值 != 1 两者不兼容。

activation: 要使用的激活函数 (详见 activations)。 如果传入 None，则不使用激活函数 (即 线性激活：a(x) = x)。

recurrent_activation: 用于循环时间步的激活函数 (详见 activations)。

use_bias: 布尔值，该层是否使用偏置向量。

kernel_initializer: kernel 权值矩阵的初始化器， 用于输入的线性转换 (详见 initializers)。

recurrent_initializer: recurrent_kernel 权值矩阵 的初始化器，用于循环层状态的线性转换 (详见 initializers)。

bias_initializer:偏置向量的初始化器 (详见initializers).

unit_forget_bias: 布尔值。 如果为 True，初始化时，将忘记门的偏置加 1。 将其设置为 True 同时还会强制 bias_initializer="zeros"。 这个建议来自 Jozefowicz et al.。

kernel_regularizer: 运用到 kernel 权值矩阵的正则化函数 (详见 regularizer)。

recurrent_regularizer: 运用到 recurrent_kernel 权值矩阵的正则化函数 (详见 regularizer)。

bias_regularizer: 运用到偏置向量的正则化函数 (详见 regularizer)。

activity_regularizer: 运用到层输出（它的激活值）的正则化函数 (详见 regularizer)。

kernel_constraint: 运用到 kernel 权值矩阵的约束函数 (详见 constraints)。

recurrent_constraint: 运用到 recurrent_kernel 权值矩阵的约束函数 (详见 constraints)。

bias_constraint: 运用到偏置向量的约束函数 (详见 constraints)。

return_sequences: 布尔值。是返回输出序列中的最后一个输出，还是全部序列。

go_backwards: 布尔值 (默认 False)。 如果为 True，则向后处理输入序列并返回相反的序列。

stateful: 布尔值 (默认 False)。 如果为 True，则批次中索引 i 处的每个样品的最后状态 将用作下一批次中索引 i 样品的初始状态。

dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于输入的线性转换。

recurrent_dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于循环层状态的线性转换。
### 输入尺寸

如果 data_format='channels_first'， 返回 5D 张量，尺寸为： (samples,time, channels, rows, cols)。

如果 data_format='channels_last'， 返回 5D 张量，尺寸为： (samples,time, rows, cols, channels)。
### 输出尺寸

如果 return_sequences，
如果 data_format='channels_first'， 返回 5D 张量，尺寸为： (samples, time, filters, output_row, output_col)。

如果 data_format='channels_last'， 返回 5D 张量，尺寸为： (samples, time, output_row, output_col, filters)。

否则，
如果 data_format ='channels_first'， 返回 4D 张量，尺寸为： (samples, filters, output_row, output_col)。

如果 data_format='channels_last'， 返回 4D 张量，尺寸为： (samples, output_row, output_col, filters)。 o_row 和 o_col 依赖于过滤器的尺寸和填充。
### 异常

ValueError: 无效的构造参数。

# SimpleRNNCell

In [ ]:
keras.layers.SimpleRNNCell(units, 
                           activation='tanh', 
                           use_bias=True, 
                           kernel_initializer='glorot_uniform', 
                           recurrent_initializer='orthogonal', 
                           bias_initializer='zeros', 
                           kernel_regularizer=None, 
                           recurrent_regularizer=None, 
                           bias_regularizer=None, 
                           kernel_constraint=None, 
                           recurrent_constraint=None, 
                           bias_constraint=None, 
                           dropout=0.0, 
                           recurrent_dropout=0.0)

# GRUCell

In [ ]:
keras.layers.GRUCell(units, 
                     activation='tanh', 
                     recurrent_activation='hard_sigmoid', 
                     use_bias=True, 
                     kernel_initializer='glorot_uniform', 
                     recurrent_initializer='orthogonal', 
                     bias_initializer='zeros', 
                     kernel_regularizer=None, 
                     recurrent_regularizer=None, 
                     bias_regularizer=None, 
                     kernel_constraint=None, 
                     recurrent_constraint=None, 
                     bias_constraint=None, 
                     dropout=0.0, 
                     recurrent_dropout=0.0, 
                     implementation=1)

# LSTMCell

In [ ]:
keras.layers.LSTMCell(units, 
                      activation='tanh', 
                      recurrent_activation='hard_sigmoid', 
                      use_bias=True, 
                      kernel_initializer='glorot_uniform', 
                      recurrent_initializer='orthogonal', 
                      bias_initializer='zeros', 
                      unit_forget_bias=True, 
                      kernel_regularizer=None, 
                      recurrent_regularizer=None, 
                      bias_regularizer=None, 
                      kernel_constraint=None, 
                      recurrent_constraint=None, 
                      bias_constraint=None, 
                      dropout=0.0, 
                      recurrent_dropout=0.0, 
                      implementation=1)

# StackedRNNCells

In [ ]:
keras.layers.StackedRNNCells(cells)

允许将一堆 RNN 单元表现为一个单元的封装器。

用于实现高效堆叠的 RNN。

### 参数

cells: RNN 单元实例的列表。

In [ ]:
cells=[
    keras.layers.LSTMCell(output_dim)
    keras.layers.LSTMCell(output_dim)
    keras.layers.LSTMCell(output_dim)
]

inputs=keras.Input((timesteps,input_dim))
x=keras.layers.RNN(cells)(inputs)

# CuDNNGRU

In [ ]:
keras.layers.CuDNNGRU(units, 
                      kernel_initializer='glorot_uniform', 
                      recurrent_initializer='orthogonal', 
                      bias_initializer='zeros', 
                      kernel_regularizer=None, 
                      recurrent_regularizer=None, 
                      bias_regularizer=None, 
                      activity_regularizer=None, 
                      kernel_constraint=None, 
                      recurrent_constraint=None, 
                      bias_constraint=None, 
                      return_sequences=False, 
                      return_state=False, 
                      stateful=False)

由 CuDNN 支持的快速 GRU 实现。

只能以 TensorFlow 后端运行在 GPU 上。

### 参数

units: 正整数，输出空间的维度。

kernel_initializer: kernel 权值矩阵的初始化器， 用于输入的线性转换 (详见 initializers)。

recurrent_initializer: recurrent_kernel 权值矩阵 的初始化器，用于循环层状态的线性转换 (详见 initializers)。

bias_initializer:偏置向量的初始化器 (详见initializers).

kernel_regularizer: 运用到 kernel 权值矩阵的正则化函数 (详见 regularizer)。

recurrent_regularizer: 运用到 recurrent_kernel 权值矩阵的正则化函数 (详见 regularizer)。

bias_regularizer: 运用到偏置向量的正则化函数 (详见 regularizer)。

activity_regularizer: Regularizer function applied to the output of the layer (its "activation"). (see regularizer).

kernel_constraint: 运用到 kernel 权值矩阵的约束函数 (详见 constraints)。

recurrent_constraint: 运用到 recurrent_kernel 权值矩阵的约束函数 (详见 constraints)。

bias_constraint: 运用到偏置向量的约束函数 (详见 constraints)。

return_sequences: 布尔值。是返回输出序列中的最后一个输出，还是全部序列。

return_state: 布尔值。除了输出之外是否返回最后一个状态。

stateful: 布尔值 (默认 False)。 如果为 True，则批次中索引 i 处的每个样品的最后状态 将用作下一批次中索引 i 样品的初始状态。

# CuDNNLSTM

In [ ]:
keras.layers.CuDNNLSTM(units, 
                       kernel_initializer='glorot_uniform', 
                       recurrent_initializer='orthogonal', 
                       bias_initializer='zeros', 
                       unit_forget_bias=True, 
                       kernel_regularizer=None, 
                       recurrent_regularizer=None, 
                       bias_regularizer=None, 
                       activity_regularizer=None, 
                       kernel_constraint=None, 
                       recurrent_constraint=None, 
                       bias_constraint=None, 
                       return_sequences=False, 
                       return_state=False, 
                       stateful=False)

由 CuDNN 支持的快速 LSTM 实现。

只能以 TensorFlow 后端运行在 GPU 上。

### 参数

units: 正整数，输出空间的维度。

kernel_initializer: kernel 权值矩阵的初始化器， 用于输入的线性转换 (详见 initializers)。

unit_forget_bias: 布尔值。 如果为 True，初始化时，将忘记门的偏置加 1。 将其设置为 True 同时还会强制 bias_initializer="zeros"。 这个建议来自 Jozefowicz et al.。

recurrent_initializer: recurrent_kernel 权值矩阵 的初始化器，用于循环层状态的线性转换 (详见 initializers)。

bias_initializer:偏置向量的初始化器 (详见initializers).

kernel_regularizer: 运用到 kernel 权值矩阵的正则化函数 (详见 regularizer)。

recurrent_regularizer: 运用到 recurrent_kernel 权值矩阵的正则化函数 (详见 regularizer)。

bias_regularizer: 运用到偏置向量的正则化函数 (详见 regularizer)。

activity_regularizer: 运用到层输出（它的激活值）的正则化函数 (详见 regularizer)。

kernel_constraint: 运用到 kernel 权值矩阵的约束函数 (详见 constraints)。

recurrent_constraint: 运用到 recurrent_kernel 权值矩阵的约束函数 (详见 constraints)。

bias_constraint: 运用到偏置向量的约束函数 (详见 constraints)。

return_sequences: 布尔值。是返回输出序列中的最后一个输出，还是全部序列。

return_state: 布尔值。除了输出之外是否返回最后一个状态。

stateful: 布尔值 (默认 False)。 如果为 True，则批次中索引 i 处的每个样品的最后状态 将用作下一批次中索引 i 样品的初始状态。